In [1]:
#Importo los módulos necesarios

from astropy.io import fits
import numpy as np
import pandas as pd


In [2]:
#Cargamos y leemos datos:

name='qsos14_added_value.fit'
hdulp=fits.open(name)
hdulp[1].header
cols = hdulp[1].columns

In [3]:
#Definimos los arrays con las columnas que vamos a usar:


logL5100=hdulp[1].data['logL5100']  #L5100 en erg/s
logL3000=hdulp[1].data['logL3000']  #L3000 en erg/s
logL_MgII=hdulp[1].data['logLMgII-BR'] #L_MgII en erg/s
FWHM_Hb = hdulp[1].data['FWHMHb-BR'] #FWHM(Hb) en km/s
FWHM_mg = hdulp[1].data['FWHMMgII-BR'] #FWHM(MgII) en km/s

#Para el CIV, probar:
logL1350=hdulp[1].data['logL1350'] #L1350 en erg/s. (Realmente según la ec. necesito L1450)
logL_CIV = hdulp[1].data['logLCIV'] #L_CIV en erg/s
FWHM_c = hdulp[1].data['FWHMCIV'] #FWHM(CIV) en km/s



logMBH= hdulp[1].data['logMBH']  #Logaritmo de la masa de los BH en Msun


In [4]:
#Hallamos L5100, L3000 y L_MgII para usarlo en las ecuaciones

L5100=[]     
for i in logL5100:
    res=10**i
    L5100.append(res)
    
L3000=[]
for i in logL3000:
    res2=10**i
    L3000.append(res2)

L_MgII=[]
for i in logL_MgII:
    res3=10**i
    L_MgII.append(res3)

#Prueba para C: hallamos L1350 y L_CIV:
L1350=[]
for i in logL1350:
    res4=10*i
    L1350.append(res4)

L_CIV=[]
for i in logL_CIV:
    res5=10*i
    L_CIV.append(i)
    
    

In [5]:
#Calclar la masa según ecuaciones del paper[2]:
    #Ecuaciones para M_BH(Hb) y M_BH(MgII)

def M_Hb(L5100,FWHM_Hb):
    
    return np.log10(1.05*(10**8)*((L5100/(10**46))**0.65)*((FWHM_Hb/(10**3))**2))

def M_mg(L3000,FWHM_mg): #Primera estimación con el MgII
    
    return np.log10(5.6*(10**6)*((L3000/(10**44))**0.62)*((FWHM_mg/(10**3))**2))

def M_MgII(L_MgII,FWHM_mg):  #Segunda estimación con el MgII
    
    return np.log10(6.79*(10**6)*((L_MgII/(10**42))**0.5)*((FWHM_mg/(10**3))**2))



####Pruebas carbono

def mu_CIV (L1350,FWHM_c): #Debería ser la línea 1450 pero no está en los datos (solo por probar)
    
    return np.log10(2.09*(10**8)*((L1350/(10**46))**0.50)*((FWHM_c/(10**3))**2))

def mu_CIV_2 (L_CIV,FWHM_c):
    
    return np.log10(2.09*(10**8)*((L_CIV/(10**46))**0.55)*((FWHM_c/(10**3))**2))


In [6]:
#Masas según estimación M_BH(Hb) y M_BH(MgII)

Mbh_hb=[]  #Masas obtenidas según las estimaciones de cada línea usando las ecuaciones anteriores. [En M_sun]
for i,j in zip(L5100,FWHM_Hb):
    masas= M_Hb(i,j)
    Mbh_hb.append(masas)
    
    
Mbh_mg=[]  #Primer método con MgII
for i,j in zip(L3000,FWHM_mg):
    masas2= M_mg(i,j)
    Mbh_mg.append(masas2)
    
Mbh_mg2=[]  #Segundo método con MgII
for i,j in zip(L_MgII,FWHM_mg):
    masas3= M_MgII(i,j)
    Mbh_mg2.append(masas3)
    
##Pruebas carbono
    
mu1_CIV=[]  
for i,j in zip(L1350,FWHM_c):
    masas4= mu_CIV(i,j)
    mu1_CIV.append(masas4)
        
mu2_CIV=[]  
for i,j in zip(L_CIV,FWHM_c):
    masas5= mu_CIV_2(i,j)
    mu2_CIV.append(masas5)

In [7]:
#Tabla para comaprar la masa obtenida con cada método con la masa de los datos.  Omitir los del C

df= pd.DataFrame({"Masa_datos": logMBH, "Mbh_Hb": Mbh_hb, "Mbh_MgII (1)": Mbh_mg, "MBh_MgII (2)": Mbh_mg2, "Mu1_CIV": mu1_CIV, "Mu2_CIV": mu2_CIV})

print(df)


        Masa_datos  Mbh_Hb  Mbh_MgII (1)  MBh_MgII (2)    Mu1_CIV    Mu2_CIV
0         8.744595     NaN      8.750739      8.695275 -12.252272 -14.976685
1         8.826519     NaN           NaN           NaN -12.025120 -14.750762
2         8.657381     NaN      8.716960      8.709823        NaN        NaN
3         9.225701     NaN           NaN           NaN -11.800715 -14.524688
4         8.502115     NaN           NaN           NaN -12.436036 -15.161685
...            ...     ...           ...           ...        ...        ...
298238    9.513645     NaN      9.666163      9.507254        NaN -15.230643
298239    8.560255     NaN      8.615667      8.520238        NaN -15.167811
298240    8.753700     NaN      8.805096      8.870440        NaN        NaN
298241    9.352207     NaN      9.486683      9.360484        NaN        NaN
298242    8.616661     NaN      9.087578      8.876493 -12.503558 -15.228632

[298243 rows x 6 columns]


In [8]:
df2 = pd.DataFrame({"Masa_datos": logMBH, "Mbh_Hb": Mbh_hb, "Mbh_MgII (1)": Mbh_mg, "MBh_MgII (2)": Mbh_mg2})
print(df2)

        Masa_datos  Mbh_Hb  Mbh_MgII (1)  MBh_MgII (2)
0         8.744595     NaN      8.750739      8.695275
1         8.826519     NaN           NaN           NaN
2         8.657381     NaN      8.716960      8.709823
3         9.225701     NaN           NaN           NaN
4         8.502115     NaN           NaN           NaN
...            ...     ...           ...           ...
298238    9.513645     NaN      9.666163      9.507254
298239    8.560255     NaN      8.615667      8.520238
298240    8.753700     NaN      8.805096      8.870440
298241    9.352207     NaN      9.486683      9.360484
298242    8.616661     NaN      9.087578      8.876493

[298243 rows x 4 columns]
